In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
import warnings
warnings.simplefilter('ignore')

In [3]:
#sub_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
filename = '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/2class/2_df_sub15.csv'
df = pd.read_csv(filename)
df = df.dropna(axis=0)
df

,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label
0,-0.178162,-0.026321,1.413727,33.535339,-2.075195,0.0
1,-0.180222,-0.026459,1.419830,33.535339,-2.064514,0.0
2,-0.183334,-0.018860,1.416016,33.532318,-2.098083,0.0
3,-0.187088,-0.015106,1.424026,33.536865,-2.069092,0.0
4,-0.186768,-0.016159,1.426697,33.541412,-2.067566,0.0
...,...,...,...,...,...,...
1563095,-0.040787,0.002884,3.567886,34.569550,-6.755066,1.0
1563096,-0.042252,-0.006912,3.589630,34.655487,-6.755066,1.0
1563097,-0.037537,-0.020325,3.575897,34.580261,-6.742859,1.0
1563098,-0.024765,-0.016434,3.578949,34.614044,-6.741333,1.0


In [4]:
X = df.drop('w_label', axis = 1)
y = df.w_label

In [5]:
X

,c_ecg,c_emg,c_eda,c_temp,c_resp
0,-0.178162,-0.026321,1.413727,33.535339,-2.075195
1,-0.180222,-0.026459,1.419830,33.535339,-2.064514
2,-0.183334,-0.018860,1.416016,33.532318,-2.098083
3,-0.187088,-0.015106,1.424026,33.536865,-2.069092
4,-0.186768,-0.016159,1.426697,33.541412,-2.067566
...,...,...,...,...,...
1563095,-0.040787,0.002884,3.567886,34.569550,-6.755066
1563096,-0.042252,-0.006912,3.589630,34.655487,-6.755066
1563097,-0.037537,-0.020325,3.575897,34.580261,-6.742859
1563098,-0.024765,-0.016434,3.578949,34.614044,-6.741333


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [8]:
check_for_nan = df.isnull().values.any()
print (check_for_nan)

False


In [9]:
import sklearn as sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [11]:
pd.Series(y).value_counts()

0.0    1082900
1.0     480200
Name: w_label, dtype: int64

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# **Random Forest**

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
forest = RandomForestClassifier(n_estimators = 50, min_samples_leaf = 5)

In [15]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train, y_train, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([29.79074025, 26.29450893, 27.47801304, 26.87776375, 27.11167312,
        26.91548896, 25.84497094, 23.17675376, 22.35960817, 24.40328884]),
 'score_time': array([0.32437372, 0.32681131, 0.32868004, 0.33710313, 0.33697414,
        0.32473063, 0.31594157, 0.32679176, 0.3101213 , 0.32600689]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

# **Random Forest(SMOTE-ENN)**

In [16]:
from imblearn.combine import SMOTEENN

SMOTEENN = SMOTEENN(random_state=0)
x_train_over,y_train_over = SMOTEENN.fit_resample(x_train,y_train)

print('SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트: ', x_train_over.shape, y_train_over.shape)
print('SMOTEENN 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트:  (937860, 5) (937860,)
SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트:  (1299990, 5) (1299990,)
SMOTEENN 적용 후 레이블 값 분포: 
 0.0    649995
1.0    649995
Name: w_label, dtype: int64


In [17]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train_over, y_train_over, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([46.50503325, 48.40967894, 47.70804524, 36.72037792, 37.43627095,
        37.27236271, 36.97075391, 41.30528069, 36.39884663, 37.98390985]),
 'score_time': array([0.42198968, 0.41917729, 0.38994122, 0.3870976 , 0.40126586,
        0.39933991, 0.40008235, 0.41040707, 0.39267874, 0.40183115]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}